
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 3.2 DEMO: Fine Grained Access Control using Dynamic Views and Partition Filtering \[Provider]

## Overview
This demonstration showcases two key data governance patterns for Delta Sharing:

- **Dynamic Views**: Row-level security based on recipient identity using `current_recipient()`
- **Partition Filtering**: Region-based data access control through table partitioning

**Provider Notebook (This Notebook):** Create source data, implement one dynamic view example and one partition filtering example, then share with recipients.

**Recipient Notebook:** Access shared views as different recipients and observe how access controls work.

### Learning Objectives
By the end of this demo, you will be able to:
1. Implement row-level security using dynamic views with `current_recipient()`
2. Create partition-based filtering for regional data boundaries
3. Share views with recipient-specific access controls

## Background

**Scenario:** You are "GlobalRetail Inc" with a sales database containing customer orders across multiple regions (North America, Europe, Asia Pacific). You need to share data with regional sales partners who should only see data from their respective regions.

In [0]:
%run ./Includes/Demo-Setup-3.2

## Example 1: Dynamic Views with Row-Level Security

Dynamic views use the `current_recipient()` function to filter data based on who is accessing it.  

<br />
<br />
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://github.com/stackql/databricks-data-sharing-and-collaboration/blob/main/images/dynamic-views.png?raw=true"
    alt="Dynamic Views"
  >
</div>
<br />
<br />

In [0]:
-- Step 1: Create source table with customer orders
CREATE OR REPLACE TABLE global_retail.sales.customer_orders (
  order_id STRING,
  customer_name STRING,
  email STRING,
  region STRING,
  product STRING,
  amount DECIMAL(10,2),
  order_date DATE
)
USING DELTA
COMMENT 'Customer orders across all regions';

In [0]:
-- Step 2: Insert sample data
INSERT INTO global_retail.sales.customer_orders VALUES
  -- North America
  ('ORD001', 'John Smith', 'john@example.com', 'North America', 'Laptop', 1200.00, '2025-01-15'),
  ('ORD002', 'Mary Johnson', 'mary@example.com', 'North America', 'Monitor', 350.00, '2025-01-16'),
  
  -- Europe  
  ('ORD003', 'Hans Mueller', 'hans@example.com', 'Europe', 'Keyboard', 89.99, '2025-01-15'),
  ('ORD004', 'Sophie Martin', 'sophie@example.com', 'Europe', 'Mouse', 45.00, '2025-01-17'),
  
  -- Asia Pacific
  ('ORD005', 'Li Wei', 'li@example.com', 'Asia Pacific', 'Tablet', 599.00, '2025-01-16'),
  ('ORD006', 'Yuki Tanaka', 'yuki@example.com', 'Asia Pacific', 'Headphones', 199.00, '2025-01-18');

SELECT * FROM global_retail.sales.customer_orders ORDER BY region;

In [0]:
-- Step 3: Create dynamic view that filters by recipient
CREATE OR REPLACE VIEW global_retail.sales.regional_orders_view AS
SELECT 
  order_id,
  customer_name,
  email,
  region,
  product,
  amount,
  order_date
FROM global_retail.sales.customer_orders
WHERE 
  CASE 
    WHEN current_recipient() = 'na_partner' THEN region = 'North America'
    WHEN current_recipient() = 'eu_partner' THEN region = 'Europe'
    WHEN current_recipient() = 'apac_partner' THEN region = 'Asia Pacific'
    WHEN current_recipient() IS NULL THEN TRUE  -- Direct query (all data)
    ELSE FALSE  -- Unknown recipient (no access)
  END;

-- When queried directly (current_recipient() is NULL), we see all records
SELECT * FROM global_retail.sales.regional_orders_view;

In [0]:
**Key Concept:** The `current_recipient()` function returns the recipient name when data is accessed through Delta Sharing, enabling automatic row-level filtering. When queried directly, it returns NULL.

## Example 2: Partition Filtering

Partition filtering leverages table partitioning to restrict data access at the storage level, providing efficient region-based access control.  

<br />
<br />
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://github.com/stackql/databricks-data-sharing-and-collaboration/blob/main/images/partition-filtering.png?raw=true"
    alt="Partition Filtering"
  >
</div>
<br />
<br />

In [0]:
-- Step 1: Create partitioned table
CREATE OR REPLACE TABLE global_retail.sales.orders_by_region (
  order_id STRING,
  customer_name STRING,
  product STRING,
  amount DECIMAL(10,2),
  order_date DATE,
  region STRING
)
USING DELTA
PARTITIONED BY (region)
COMMENT 'Orders partitioned by region for efficient filtering';

-- Step 2: Insert data (automatically partitioned by region)
INSERT INTO global_retail.sales.orders_by_region VALUES
  ('ORD001', 'John Smith', 'Laptop', 1200.00, '2025-01-15', 'North America'),
  ('ORD002', 'Mary Johnson', 'Monitor', 350.00, '2025-01-16', 'North America'),
  ('ORD003', 'Hans Mueller', 'Keyboard', 89.99, '2025-01-15', 'Europe'),
  ('ORD004', 'Sophie Martin', 'Mouse', 45.00, '2025-01-17', 'Europe'),
  ('ORD005', 'Li Wei', 'Tablet', 599.00, '2025-01-16', 'Asia Pacific'),
  ('ORD006', 'Yuki Tanaka', 'Headphones', 199.00, '2025-01-18', 'Asia Pacific');

-- View data by partition
SELECT * FROM global_retail.sales.orders_by_region WHERE region = 'Europe';

In [0]:
-- Step 3: Create partition-specific shares
-- Share only North America partition
CREATE SHARE IF NOT EXISTS na_orders_share
COMMENT 'North America orders only';

ALTER SHARE na_orders_share 
ADD TABLE global_retail.sales.orders_by_region 
PARTITION (region = 'North America');

-- Share only Europe partition  
CREATE SHARE IF NOT EXISTS eu_orders_share
COMMENT 'Europe orders only';

ALTER SHARE eu_orders_share 
ADD TABLE global_retail.sales.orders_by_region 
PARTITION (region = 'Europe');

SHOW SHARES;

In [0]:
**Key Concept:** Partition filtering shares specific partitions of a table with recipients. Recipients can only access data in the shared partitions, providing storage-level access control.

In [0]:
## Setup Recipients and Grant Access

In [0]:
-- Create recipients
CREATE RECIPIENT IF NOT EXISTS na_partner
COMMENT 'North America regional partner';

CREATE RECIPIENT IF NOT EXISTS eu_partner
COMMENT 'Europe regional partner';

CREATE RECIPIENT IF NOT EXISTS apac_partner
COMMENT 'Asia Pacific regional partner';

-- Create share for dynamic views
CREATE SHARE IF NOT EXISTS regional_orders_dynamic
COMMENT 'Orders with dynamic view filtering';

ALTER SHARE regional_orders_dynamic 
ADD VIEW global_retail.sales.regional_orders_view;

-- Grant access to all partners (filtering handled by view logic)
GRANT SELECT ON SHARE regional_orders_dynamic TO RECIPIENT na_partner;
GRANT SELECT ON SHARE regional_orders_dynamic TO RECIPIENT eu_partner;
GRANT SELECT ON SHARE regional_orders_dynamic TO RECIPIENT apac_partner;

In [0]:
-- Grant access to partition-specific shares
GRANT SELECT ON SHARE na_orders_share TO RECIPIENT na_partner;
GRANT SELECT ON SHARE eu_orders_share TO RECIPIENT eu_partner;

-- Show all shares and recipients
SELECT 'Shares' as type, name, comment FROM (SHOW SHARES)
UNION ALL
SELECT 'Recipients' as type, name, comment FROM (SHOW RECIPIENTS);

## Summary

✅ **What we accomplished:**

**Example 1 - Dynamic Views:**
- Created a view using `current_recipient()` function
- Implemented row-level security that filters data based on who is accessing it
- All partners access the same share but see different data

**Example 2 - Partition Filtering:**
- Created a partitioned table by region
- Shared specific partitions with specific recipients
- Recipients can only access their designated partition

**Comparison:**
| Pattern | Granularity | Flexibility | Performance |
|---------|-------------|-------------|-------------|
| Dynamic Views | Row-level | High (logic-based) | Good for moderate data |
| Partition Filtering | Partition-level | Low (structural) | Excellent (storage-level) |

**Next Steps:** Continue to the recipient notebook to see how these access controls work from the recipient's perspective.

&copy; 2025 Databricks, Inc. All rights reserved.